In [3]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects,SDS_islands,SDS_universal_transects

In [4]:
# region of interest (longitude, latitude in WGS84), can be loaded from a .kml polygon
polygon = SDS_tools.polygon_from_kml(os.path.join(os.getcwd(), 'example','Fuvahmulah.kml'))
# date range
dates = ['2024-04-27', '2024-08-20']#['2018-01-01', '2024-03-01']
# satellite missions
sat_list = ['S2']#,'L8']
# choose Landsat collection 'C01' or 'C02'
collection = 'C02'
# name of the site
sitename = 'Fuvahmulah'
# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data')
# put all the inputs into a dictionnary

inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath,
         'landsat_collection': collection}

In [5]:
# before downloading the images, check how many images are available for your inputs
SDS_download.check_images_available(inputs);

Number of images available between 2024-04-27 and 2024-08-20:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     S2: 13 images
  Total to download: 13 images


In [6]:
# metadata = SDS_download.retrieve_images(inputs)

In [7]:
data_folder = 'data\\Fuvahmulah\\S2\\'
for root, dirs, files in os.walk(data_folder):
    for file in files:
        if 'dup' in file:
            os.remove(os.path.join(root, file))
            print(file)

**If you have already retrieved the images**, just load the metadata file by only running the section below

In [8]:
metadata = SDS_download.get_metadata(inputs) 

In [28]:
# settings for the sand contour mapping
settings = { 
    # general parameters:
    'cloud_thresh': 0.2,#0.5,        # threshold on maximum cloud cover
    'dist_clouds': 10,#300,         # ditance around clouds where shoreline can't be mapped
    'output_epsg': 4326,#3857,        # epsg code of spatial reference system desired for the output
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold        
    'check_detection_sand_poly': False, # if True, uses sand polygon for detection and shows user for validation 
    'save_figure': True,               # if True, saves a figure showing the mapped shoreline for each image
    # add the inputs defined previously
    'inputs': inputs,
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 10,        # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 20,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 0.1,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': True,#False,  # switch this parameter to True if sand pixels are masked (in black) on many images
    'sand_color': 'bright',#'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off': False,           # True to switch pansharpening off for Landsat 7/8/9 imagery
    's2cloudless_prob': 60,     # probability threshold to identify cloudy pixels in the s2cloudless mask
}



### [OPTIONAL] Save .jpg of the satellite images 
Saves .jpg files of the preprocessed satellite images (cloud masking + pansharpening/down-sampling) under *./data/sitename/jpeg_files\preprocessed*

In [21]:
SDS_preprocess.save_jpg(metadata, settings)

Saving images as jpg:
S2: 477 images
100%
Satellite images saved as .jpg in d:\Github\CoastSat\data\Fuvahmulah\jpg_files\preprocessed


In [14]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)
settings['max_dist_ref'] = 100 # max distance (in meters) allowed from the reference shoreline

(29, 2)
Reference shoreline has been saved in d:\Github\CoastSat\data\Fuvahmulah


In [36]:
#%matplotlib qt
output = SDS_shoreline.extract_shorelines(metadata, settings)
output

Mapping shorelines:
S2:   100%sand detected, enhanced contours
Backend Qt5Agg is interactive backend. Turning interactive mode on.
[]



{'dates': [datetime.datetime(2015, 8, 16, 5, 37, 6, tzinfo=<UTC>)],
 'shorelines': [array([], shape=(0, 2), dtype=float64)],
 'filename': ['2015-08-16-05-37-06_S2_Fuvahmulah_ms.tif'],
 'cloud_cover': [0.15273393658810325],
 'geoaccuracy': ['PASSED'],
 'idx': [0],
 'MNDWI_threshold': [-0.04481933795435766],
 'satname': ['S2']}

In [31]:
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
    output = pickle.load(f)
# remove duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_duplicates(output)
# remove inaccurate georeferencing (set threshold to 10 m)
output = SDS_tools.remove_inaccurate_georef(output, 10)

0 duplicates
0 bad georef


In [32]:
shorelines = output['shorelines']
shorelines

[array([], shape=(0, 2), dtype=float64)]

In [19]:
#export boundaries as csv
shoreline_path = 'data/Fuvahmulah/Fuvahmulah_output2.csv'
shorelines = output['shorelines']
dates = output['dates']
for i in range(len(shorelines)):
    distances = [datetime.strftime(dates[i],'%Y-%m-%d')] 
    for j in range(len(shorelines[i])):
        distances.append(str(shorelines[i][j])) 
    SDS_universal_transects.add_line_to_csv(distances,shoreline_path)

SDS_universal_transects.infill_missing_days(shoreline_path)